# A little about Numba

C/C++ performance in a Python Library

[Numba Github](https://github.com/numba/numba)

### Python Speed Comparision

<img src=LangSpeeds.png>

[Language Speed Benchmarks](https://julialang.org/benchmarks/)

# Rolling Dice
Rolling two dice, one million times, counting the number of times you get double sixes

In [1]:
from random import randrange, random

def dice():
    count = 0
    roll = 0

    while(roll <= 1000000):
        d1 = randrange(1,7)
        d2 = randrange(1,7)

        roll += 1
        if d1 == 6 and d2 == 6:
            count += 1

    #print("# of times double 6's were rolled:", count)|
    return count/1000000 #probability of rolling both 6's 

In [2]:
%time dice() #1/36

Wall time: 1.43 s


0.027935

In [3]:
from numba import jit

@jit
def accel_dice():
    count = 0
    roll = 0

    while(roll <= 1000000):
        d1 = randrange(1,7)
        d2 = randrange(1,7)

        roll += 1
        if d1 == 6 and d2 == 6:
            count += 1
    #print("# of times double 6's were rolled:", count)
    return count/1000000

In [4]:
%time accel_dice()

Wall time: 284 ms


0.02788

In [5]:
import random

@jit(nopython=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

def monte_carlo_pi_no_numba(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [6]:
%timeit monte_carlo_pi_no_numba(100000)

Wall time: 60.6 ms


3.14408

In [10]:
%timeit monte_carlo_pi(100000)

1.15 ms ± 29.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Just In Time compilation

<img src="jitcompile.png">


In [8]:
import dis

dis.dis(monte_carlo_pi)

  5           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (acc)

  6           4 LOAD_GLOBAL              0 (range)
              6 LOAD_FAST                0 (nsamples)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                48 (to 62)
             14 STORE_FAST               2 (i)

  7          16 LOAD_GLOBAL              1 (random)
             18 LOAD_METHOD              1 (random)
             20 CALL_METHOD              0
             22 STORE_FAST               3 (x)

  8          24 LOAD_GLOBAL              1 (random)
             26 LOAD_METHOD              1 (random)
             28 CALL_METHOD              0
             30 STORE_FAST               4 (y)

  9          32 LOAD_FAST                3 (x)
             34 LOAD_CONST               2 (2)
             36 BINARY_POWER
             38 LOAD_FAST                4 (y)
             40 LOAD_CONST               2 (2)
             42 BINARY_P

# Whats the Catch?

[Supported Python](https://numba.pydata.org/numba-doc/dev/reference/pysupported.html)

In [11]:
@jit
def jit_dict(x):
    return x['key']

jit_dict(dict(key='value'))

<ipython-input-11-8787d4587746>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "jit_dict" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-11-8787d4587746> (3)

File "<ipython-input-11-8787d4587746>", line 3:
def jit_dict(x):
    return x['key']
    ^

  @jit
C:\Users\thaih\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:177: NumbaWarning: Function "jit_dict" was compiled in object mode without forceobj=True.

File "<ipython-input-11-8787d4587746>", line 2:
@jit
def jit_dict(x):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\thaih\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-o

'value'

In [14]:
from numba import njit

@njit
def nopython(x):
    return x['key']

nopython(dict(key='value'))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at <ipython-input-14-f22bc8fbc4ac> (5)[0m
[1m
File "<ipython-input-14-f22bc8fbc4ac>", line 5:[0m
[1mdef nopython(x):
[1m    return x['key']
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class 'dict'>[0m


# Let's throw in some GPU

In [15]:
import numpy as np
from numba import vectorize

nums = np.arange(10)

@vectorize
def add_ten(num):
    return num + 10 

In [16]:
 add_ten(nums)

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype=int64)

In [17]:
@vectorize(['int64(int64, int64)'], target='cuda') 
def add_ufunc(x, y):
    return x + y

In [18]:
a = np.array([10, 20, 30, 40])
b = np.arange(4*4).reshape((4,4))

In [19]:
%timeit np.add(a,b)

1.21 µs ± 44.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [20]:
%timeit add_ufunc(a,b)

1.43 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Some Speed Comparisons

In [21]:
@vectorize
def cpu_gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [22]:
import math # Note that for the CUDA target, we need to use the scalar functions from the math module, not NumPy

SQRT_2PI = np.float32((2*math.pi)**0.5)  # Precompute this constant as a float32.  Numba will inline it at compile time.

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gpu_gaussian_pdf(x, mean, sigma):
    '''Compute the value 
    of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [23]:
# Evaluate the Gaussian a million times!
x = np.random.uniform(-3, 3, size=1000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

# Quick test on a single element just to make sure it works
gpu_gaussian_pdf(x[0], 0.0, 1.0)

array([0.04451605], dtype=float32)

In [24]:
import scipy.stats # for definition of gaussian distribution, so we can compare CPU to GPU time
scipy_pdf = scipy.stats.norm

In [25]:
%timeit cpu_gaussian_pdf(x, mean, sigma)

12.5 ms ± 407 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%timeit gpu_gaussian_pdf(x, mean, sigma)

6.95 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit scipy_pdf.pdf(x, loc=mean, scale=sigma)

72.1 ms ± 5.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Conclusion

Numba shines when you code is:

+ numerically oriented 
  - int
  - float
  - complex
+ using lots of numpy
+ using lots of functions
+ using lots of loops

# Decay of an Isotope

Exponential decay formula: $N(t) = N(0) 2^{-t/\tau}$

Probability of decay: $p(t) = 1 - 2^{-t/\tau}$

Simulate decay of 1000 atoms. 


In [ ]:
from numpy import arange
import matplotlib.pyplot as plt

In [ ]:
def N_decay(N,tau,tmax,t_step,graph):
    N_t = 0
    p = 1 - 2**(-t_step/tau)
    
    tpoints = arange(0.0,tmax,t_step)
    N_arr = []
    N_t_arr = []

    for t in tpoints:
        N_arr.append(N)
        N_t_arr.append(N_t)
        #print(N_arr)
        decay = 0
        for i in range(N):
            if random() < p:
                decay+=1
                #print(decay)
        N -= decay
        N_t += decay
        
    if graph == 1:
        plt.plot(tpoints,N_arr)
        plt.plot(tpoints,N_t_arr)

In [ ]:
#Values for decay of Thallium 208 to Lead 208
N = 1000
tau = 3.053*60
tmax = 1000
t_step = 1.0

In [ ]:
N_decay(N,tau,tmax,t_step,1)

In [ ]:
%timeit N_decay(N,tau,tmax,t_step,0)

In [ ]:
#Throw something crazy in...
N_m = 1000000
tau = 3.053*60
tmax_m = 1000
t_step_m = 1.0

In [ ]:
N_decay(N_m,tau,tmax_m,t_step,1)

In [ ]:
%timeit N_decay(N_m,tau,tmax_m,t_step,0)

## Let's speed it up...

In [ ]:
@jit
def N_decay(N,tau,tmax,t_step,graph):
    N_t = 0
    p = 1 - 2**(-t_step/tau)
    
    tpoints = arange(0.0,tmax,t_step)
    N_arr = []
    N_t_arr = []

    for t in tpoints:
        N_arr.append(N)
        N_t_arr.append(N_t)
        #print(N_arr)
        decay = 0
        for i in range(N):
            if random() < p:
                decay+=1
                #print(decay)
        N -= decay
        N_t += decay
        
    if graph == 1:
        plt.plot(tpoints,N_arr)
        plt.plot(tpoints,N_t_arr)

In [ ]:
%timeit N_decay(N_m,tau,tmax_m,t_step,0)